# Group work - Classification

In this assignment, we will focus on sports analytics. This data set is made available by http://www.baseball-reference.com. It contains data about professional baseball (MLB) games played in the 2016 season. There are 2,427 games in the data set. Each row represents a single game. The goal is to predict the attendance at a home team’s game. This is an important task because most franchises want to predict the number of attendees for a variety of reasons including profits.

## Description of Variables

The description of variables are provided in "Baseball - Data Dictionary.docx"

## Goal

Use the **baseball.csv** data set and build a model to predict **attendance_binary**.

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


## Recommended roles for group members:

**Section 1:** to be completed by both group members

**Section 2:** to be completed by the first group member and checked by the second

**Section 3:** to be completed by the second group member and checked by the first

**Important notes:**
- Both group members will get the same grade. Therefore, you should check the work of your group member. If they make a mistake, you will be responsible for that mistake too.
- Both group members must put in their fair share of effort. Otherwise, those who don't contribute to the assignment will not receive any grade.


# Section 1: (6 points in total)

## Data Prep (5.5 points)

In [161]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)

In [162]:
# Get the data
# We will be predicting the attendance_binary variable
baseball = pd.read_csv(r"D:\unknown\Education\Spring 22\DSP\Assignment 1\baseball.csv")
baseball.head()

,attendance_binary,previous_attendance,previous_away_team_errors,previous_away_team_hits,previous_away_team_runs,game_type,previous_game_type,previous_home_team_errors,previous_home_team_hits,previous_home_team_runs,game_day,previous_game_day,temperature,wind_speed,sky,previous_game_duration,previous_homewin
0,0,43683,2,6,2,Night Game,Day Game,0,6,6,Wednesday,Monday,55,24,Overcast,2.933333,1
1,0,45785,0,7,2,Night Game,Day Game,0,10,3,Wednesday,Monday,48,7,Unknown,2.800000,1
2,0,48282,0,8,4,Night Game,Day Game,2,4,3,Wednesday,Monday,65,10,Cloudy,3.383333,0
3,0,21830,0,9,6,Day Game,Night Game,0,15,11,Wednesday,Tuesday,77,0,In Dome,3.233333,1
4,0,49289,2,4,2,Night Game,Day Game,1,1,3,Tuesday,Monday,81,12,Cloudy,2.633333,1


# Splitting the data into train and test

In [163]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(baseball, test_size=0.3)

# checking the missing values

In [164]:
train_set.isna().sum()

attendance_binary            0
previous_attendance          0
previous_away_team_errors    0
previous_away_team_hits      0
previous_away_team_runs      0
game_type                    0
previous_game_type           0
previous_home_team_errors    0
previous_home_team_hits      0
previous_home_team_runs      0
game_day                     0
previous_game_day            0
temperature                  0
wind_speed                   0
sky                          0
previous_game_duration       0
previous_homewin             0
dtype: int64

In [165]:
test_set.isna().sum()

    

attendance_binary            0
previous_attendance          0
previous_away_team_errors    0
previous_away_team_hits      0
previous_away_team_runs      0
game_type                    0
previous_game_type           0
previous_home_team_errors    0
previous_home_team_hits      0
previous_home_team_runs      0
game_day                     0
previous_game_day            0
temperature                  0
wind_speed                   0
sky                          0
previous_game_duration       0
previous_homewin             0
dtype: int64

In [166]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [167]:
## Separate the target variable and dropping them from the dataset

In [222]:
train_y = train_set['attendance_binary']
test_y = test_set['attendance_binary']

train_inputs = train_set.drop(['attendance_binary'], axis=1)
test_inputs = test_set.drop(['attendance_binary'], axis=1)

In [169]:
##Identify the numerical and categorical columns

In [170]:
train_inputs.dtypes

previous_attendance            int64
previous_away_team_errors      int64
previous_away_team_hits        int64
previous_away_team_runs        int64
game_type                     object
previous_game_type            object
previous_home_team_errors      int64
previous_home_team_hits        int64
previous_home_team_runs        int64
game_day                      object
previous_game_day             object
temperature                    int64
wind_speed                     int64
sky                           object
previous_game_duration       float64
previous_homewin               int64
dtype: object

In [171]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [172]:
#From the data.head() we can see that one column previous_homewin is in binary so we'll have to convert it.
binary_columns= ['previous_homewin']
for col in binary_columns:
    numeric_columns.remove(col)

## Pipeline


In [174]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())])

In [175]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [176]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [177]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')



In [178]:
train_x=preprocessor.fit_transform(train_inputs)
train_x

array([[-1.12371621,  0.57666325, -0.20719118, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.05787587, -0.72716391, -0.78017264, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38394295,  0.57666325, -1.35315411, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.52534761,  0.57666325, -0.78017264, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.94392488, -0.72716391,  0.65228102, ...,  0.        ,
         0.        ,  0.        ],
       [-0.98407336,  1.88049041, -0.49368191, ...,  1.        ,
         0.        ,  1.        ]])

In [179]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.0814842 ,  1.88049041, -1.06666338, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.17001331, -0.72716391, -1.63964484, ...,  0.        ,
         0.        ,  0.        ],
       [-0.29905768, -0.72716391, -0.20719118, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.49582715, -0.72716391,  1.22526249, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.26445059, -0.72716391, -0.20719118, ...,  0.        ,
         0.        ,  1.        ],
       [-1.71775245, -0.72716391, -1.63964484, ...,  0.        ,
         0.        ,  0.        ]])

## Find the Baseline (0.5 point)
# Baseline Accuracy


In [180]:
# Find majority class
train_y.value_counts()

attendance_binary
1                    873
0                    825
dtype: int64

In [181]:
# Find percentage
train_y.value_counts()/len(train_y)

attendance_binary
1                    0.514134
0                    0.485866
dtype: float64

# Section 2: (3 points in total)

Build three different SVM models (by changing the kernels, regularization, etc.). Generate their training and test values. Each model is worth 1 point. 

(Add cells as needed)

## SVM Model 1:

In [229]:
from sklearn.svm import LinearSVC

svm_clf = LinearSVC(C=10, max_iter=100000) 
#We can also slove 'ConvergenceWarning: Liblinear failed to converge' warning by setting 'dual = False'

svm_clf.fit(train_x, train_y)

LinearSVC(C=10, max_iter=100000)

## Accuracy

In [230]:
from sklearn.metrics import accuracy_score

#Predict the train values
train_y_pred = svm_clf.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8445229681978799

In [231]:
#Predict the test values
test_y_pred = svm_clf.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8148148148148148

In [232]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[278,  65],
       [ 70, 316]], dtype=int64)

In [233]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.80      0.81      0.80       343
           1       0.83      0.82      0.82       386

    accuracy                           0.81       729
   macro avg       0.81      0.81      0.81       729
weighted avg       0.82      0.81      0.81       729



## SVM Model 2:

In [234]:
from sklearn.svm import SVC

pol_svm2 = SVC(kernel="poly", degree=2, coef0=1, C=1, gamma='scale')

pol_svm2.fit(train_x, train_y)

SVC(C=1, coef0=1, degree=2, kernel='poly')

In [235]:
#Predict the train values
train_y_pred = pol_svm2.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8733804475853946

In [236]:
#Predict the test values
test_y_pred = pol_svm2.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8244170096021948

In [237]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[283,  60],
       [ 68, 318]], dtype=int64)

In [238]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       343
           1       0.84      0.82      0.83       386

    accuracy                           0.82       729
   macro avg       0.82      0.82      0.82       729
weighted avg       0.82      0.82      0.82       729



## SVM Model 3:SVC(kernel='rbf')

In [250]:
rbf_svm = SVC(kernel="rbf", C=1, gamma='scale')

rbf_svm.fit(train_x, train_y)

SVC(C=1)

In [251]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8851590106007067

In [252]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.821673525377229

In [254]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[280,  63],
       [ 67, 319]], dtype=int64)

In [255]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       343
           1       0.84      0.83      0.83       386

    accuracy                           0.82       729
   macro avg       0.82      0.82      0.82       729
weighted avg       0.82      0.82      0.82       729



# Section 3: (3 points in total)

Build two different SGD models (by changing the penalty, etc. or adding polynomial terms) and one LogisticRregression model. Generate their training and test values. Each model is worth 1 point.

(Add cells as needed)

## SGD Model 1:

In [258]:
from sklearn.linear_model import SGDClassifier 

# tol = stopping criterion
# eta0 = learning rate
# penalty = regularization term
# max_iter = number of passes over training data (i.e., epochs)

sgd_logreg = SGDClassifier(max_iter=100, penalty=None, eta0=0.1, tol=0.0001) 

sgd_logreg.fit(train_x, train_y)

SGDClassifier(eta0=0.1, max_iter=100, penalty=None, tol=0.0001)

In [259]:
sgd_reg.n_iter_

16

In [269]:
#Predict the train values
train_y_pred = sgd_logreg.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.833922261484099

In [270]:
#Predict the test values
test_y_pred = sgd_logreg.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.7928669410150891

In [271]:
confusion_matrix(test_y, test_y_pred)

array([[261,  82],
       [ 69, 317]], dtype=int64)

In [272]:
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.79      0.76      0.78       343
           1       0.79      0.82      0.81       386

    accuracy                           0.79       729
   macro avg       0.79      0.79      0.79       729
weighted avg       0.79      0.79      0.79       729



## SGD Model 2:

In [197]:
### Stochastic Gradient Descent version with penalty

In [277]:
sgd_logreg2 = SGDClassifier(max_iter=100, penalty='l2', eta0=0.1, tol=0.0001) 

sgd_logreg2.fit(train_x, train_y)

SGDClassifier(eta0=0.1, max_iter=100, tol=0.0001)

In [278]:
#Predict the train values
train_y_pred = sgd_logreg2.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8103651354534747

In [279]:
#Predict the test values
test_y_pred = sgd_logreg2.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8106995884773662

In [280]:
confusion_matrix(test_y, test_y_pred)

array([[270,  73],
       [ 65, 321]], dtype=int64)

In [281]:
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80       343
           1       0.81      0.83      0.82       386

    accuracy                           0.81       729
   macro avg       0.81      0.81      0.81       729
weighted avg       0.81      0.81      0.81       729



## LogisticRegression Model:

In [283]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()

log_reg.fit(train_x, train_y)

LogisticRegression()

In [202]:
## Accuracy

In [203]:
from sklearn.metrics import accuracy_score

In [284]:
#Predict the train values
train_y_pred = log_reg.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8433451118963486

In [285]:
#Predict the test values
test_y_pred = log_reg.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.813443072702332

In [286]:
from sklearn.metrics import confusion_matrix

#We usually create the confusion matrix on test set
confusion_matrix(test_y, test_y_pred)

array([[279,  64],
       [ 72, 314]], dtype=int64)

In [287]:
from sklearn.metrics import classification_report

#We usually create the classification report on test set
print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       343
           1       0.83      0.81      0.82       386

    accuracy                           0.81       729
   macro avg       0.81      0.81      0.81       729
weighted avg       0.81      0.81      0.81       729



# Discussion (3 points in total)


## List the train and test values of each model you built (1 point)

## Which model performs the best and why? (0.5 points) How does it compare to baseline? (0.5 points)

Hint: The best model is the one that has the highest TEST score (regardless of any of the training values). If you select your model based on TRAIN values, you will lose points.

## Is there any evidence of overfitting in the best model, why or why not? If there is, what did you do about it? (0.5 points)

## Is there any evidence of overfitting in the other models (besides the best model), why or why not? If there is, what did you do about it? (0.5 points)